# 05 Related vs unrelated transfer
Hypothesis: related source-target pairs are both more accurate and more stable.

### Expected Outcome
Related transfer should dominate unrelated transfer under identical adaptation policy.

In [ ]:
from pathlib import Path
import sys
import pandas as pd
import matplotlib.pyplot as plt

ROOT = Path.cwd().resolve()
while ROOT != ROOT.parent and not (ROOT / 'src').is_dir():
    ROOT = ROOT.parent
sys.path.insert(0, str(ROOT / 'src'))

from utils.notebook_transfer import TransferNotebookLab

LAB = TransferNotebookLab.from_root(ROOT)
FAST_DEV_RUN = False

In [ ]:
method = 'gradual_unfreeze'

LAB.run(
    base_name='transfer_core_related.yaml',
    notebook_tag='05_related',
    fast_dev_run=FAST_DEV_RUN,
    overrides={'methods': [method]},
)
related = LAB.read_method(method).assign(scenario='related')

LAB.run(
    base_name='transfer_unrelated_failure.yaml',
    notebook_tag='05_unrelated',
    fast_dev_run=FAST_DEV_RUN,
    overrides={'methods': [method]},
)
unrelated = LAB.read_method(method).assign(scenario='unrelated')

both = pd.concat([related, unrelated], ignore_index=True)
both.head()

In [ ]:
fig, ax = plt.subplots(figsize=(6.6, 3.7))
for scenario, df in both.groupby('scenario'):
    ax.plot(df['epoch'], df['target_test_acc'], marker='o', label=scenario)
ax.set_title('Related vs unrelated target accuracy')
ax.set_xlabel('epoch')
ax.set_ylabel('target_test_acc')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '05_related_vs_unrelated_target_acc.png')

fig, ax = plt.subplots(figsize=(6.6, 3.7))
for scenario, df in both.groupby('scenario'):
    ax.plot(df['epoch'], df['feature_drift'], marker='o', label=scenario)
ax.set_title('Related vs unrelated feature drift')
ax.set_xlabel('epoch')
ax.set_ylabel('feature_drift')
ax.grid(alpha=0.25)
ax.legend(frameon=False)
LAB.savefig(fig, '05_related_vs_unrelated_feature_drift.png')

In [ ]:
summary = both.groupby('scenario').agg(
    final_target_acc=('target_test_acc', 'last'),
    final_retention=('source_retention_acc', 'last'),
    final_drift=('feature_drift', 'last'),
).reset_index()
summary

### Interpretation
Relatedness appears in full trajectories, not only endpoints.